In [1]:
!pip3 install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd  #for dataset

import numpy as np   # for matrices and array

from transformers import AutoTokenizer # trnasformer is a library and AutoTokenizer is a part of it.
# Tokenizer => To break a sentence into words is called tokenization.It is done so that the data can be easily understood by the model.
# Ex : I am with 5th Sem students of KIET. => AFter tokenization : [I,am,with,5th,Sem,students,of,KIET]
# Natural language data is the hardest to train
# Autotokenizer is a program which as great functions to tokenize such type of hard data
# Tokenizers are also trained ml models to do tokenization

import torch 
from indicnlp.tokenize import indic_tokenize



In [3]:
data = pd.read_csv("/kaggle/input/english-hindi/Sentence pairs in English-Hindi - 2025-02-11.tsv",
                  sep="\t", header=None, names=["SrcSentID","SrcSent","DstSentID","DstSent"])

In [4]:
data.drop(labels=[data.columns[0], data.columns[2]], axis=1, inplace=True)
# This line removes the first and third columns of the dataframe, permanently updating data.

In [5]:
data.head()

,SrcSent,DstSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [6]:
# to tokenize english sentence we will use AutoTokenizer 
# for hindi => indic_tokenize
src_sent_tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base") # T5 is an LLM based on tenasformers made by google
# we are using base model of T5
# "google-T5/T5-base" => It is like a path to a repository on internet => hugging face ki ek repository hogi uske andr ggoogle-T5 ek repository hogi jiske andr ek folder hoga T5-base
# hr LLM ka Apna ek tokenizer hota hai jo ki ek alg trh se train kiya jata hai


config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
data["SrcSent"] = data["SrcSent"].apply(lambda x : src_sent_tokenizer.tokenize(x))
# hr cell pr kch operation krna hai to applymap use krenge
# kisi ek row ya column  pr opr krna hai to yse apply
# agr hashing krni hai to t=use map
# panda has => apply, map, applymap



In [8]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [9]:
# this type of tokenization is called sub-word tokenization as it also breaks a single word into parts and it also treart space as a token represnted by unserscore(_)
# BPE(Byte-Pair encoding) => Algorithm is used behind this to do this type of tokenization
# WPT(Word Piece Tokenizer) => it is also an algo for tokenization

In [10]:
data["DstSent"] = data["DstSent"].apply(lambda x : indic_tokenize.trivial_tokenize(x,lang="hi"))

In [11]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [12]:
data["SrcSent"] = data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [13]:
Vs = src_sent_tokenizer.get_vocab() # hr src word ko ek number assign kr dega

In [14]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [15]:
hindi_vocab = set()

for tokenized_hindi_sent in data["DstSent"] :
    hindi_vocab.update(tokenized_hindi_sent) # this is a set of unique hindi words from our dst column

In [16]:
Vd = dict()
for idx, token in enumerate(hindi_vocab) : 
    Vd[token] = idx + 3
Vd["<PAD>"] = 0
Vd["<SOS>"] = 1
Vd["<EOS>"] = 2

Tokens in neural network: 
"<SOS>" (1) => Start of Sentence => USed to denote or tell the neural network that a sentece is going to start.
"<PAD>" (0) => Padding => Used for doing padding to make the length of all the sentence equal, so that a matrix can be made"
"<EOS>" (2) => End of Service => USed to denote the end of the sentence.

In [17]:
def convert_hindi_tokens_to_ids(hindi_sent) : 
    return [Vd[token] for token in hindi_sent]

In [18]:
data["DstSent"] = data["DstSent"].apply(lambda x : convert_hindi_tokens_to_ids(x))

In [19]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4279, 498, 1072, 4466, 6787, 5176, 5718, 4891..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4279, 498, 1072, 4466, 6787, 4891, 963]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[3804, 5349, 2896, 400, 4475, 5082, 2774, 5038..."
3,"[466, 751, 31, 17, 1837, 5]","[1844, 1049, 5654, 963]"
4,"[27, 3041, 25, 5]","[146, 6721, 5077, 2290, 5716, 4891, 963]"


In [20]:
# adding SOS(1) in start
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids # Concatenation hora h

In [21]:
data["DstSentInput"] = data["DstSent"].apply(lambda x : insert_sos_token_id(x))

In [22]:
 # adding EOS (2) at end
def insert_eos_token_id(hindi_sent_token_ids): 
    return hindi_sent_token_ids + [2]

In [23]:
data["DstSentLabel"] = data["DstSent"].apply(lambda x : insert_eos_token_id(x))

In [24]:
data.drop(labels=[data.columns[1]],axis=1,inplace=True)

In [25]:
X = list(data["SrcSent"])
Y_input = list(data["DstSentInput"])
Y_label = list(data["DstSentLabel"])

In [26]:
X_tensor = [torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in X]
Y_input_tensor = [torch.tensor(tokenized_hin_sent_ids) for tokenized_hin_sent_ids in Y_input]
Y_label_tensor = [torch.tensor(tokenized_hin_sent_ids) for tokenized_hin_sent_ids in Y_label]

In [27]:
X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_padded_input = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [28]:
Ns = X_padded.shape[1]
Nd = Y_padded_label.shape[1]

In [29]:
# Making a class Encoder and inheriting a parent class torch.nn.module
# We have made the layers of encoders but not connected them yet
# forward is function from the parent class that is Module and we are inherriting and overwriting that function so it can not be named anything else.
# forward function me hmne connection kiya hai
class Encoder(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,hidden_size=word_embedding_dim,batch_first=True)

    def forward(self,X_padded_mini_batch):
        first_embedding_layer_out = self.first_embedding_layer(X_padded_mini_batch)
        encoder_output, (final_encoder_output,final_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_output, (final_encoder_output,final_cell_state)

        

In [30]:
# word_embedding_size in encoder and decoder should be same
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_features=word_embedding_dim,out_features=dst_lang_vocab_size)
        self.prediction_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_input_mini_batch)
        decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out,
                                                                                                         (final_encoder_output,
                                                                                                          final_cell_state))
        prediction = self.prediction_layer_activation(self.prediction_layer(decoder_lstm_layer_out))
        
        return prediction, (final_decoder_lstm_layer_out, final_cell_state)

In [31]:
class Seq2SeqEncDec(torch.nn.Module):
    def __init__(self,src_lang_vocab_size, dst_lang_vocab_size, word_embedding_dim):
        super(Seq2SeqEncDec, self).__init__()

        self.encoder = Encoder(src_lang_vocab_size,word_embedding_dim)
        self.decoder = Decoder(dst_lang_vocab_size,word_embedding_dim)

    def forward(self,X_padded_mini_batch,Y_padded_input_mini_batch):
        encoder_output, (final_encoder_output,final_cell_state) = self.encoder(X_padded_mini_batch)
        y_hat_mini_batch = self.decoder(Y_padded_input_mini_batch, final_encoder_output,final_cell_state)

        return y_hat_mini_batch

Gradient Descent is a covex optimization algo and it is unconstrained

In [32]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [33]:
X_padded_train = X_padded[0:13000]
Y_padded_input_train = Y_padded_input[0:13000]
Y_padded_label_train = Y_padded_label[0:13000]

X_padded_test = X_padded[13000:]
Y_padded_input_test = Y_padded_input[13000:]
Y_padded_label_test = Y_padded_label[13000:]

In [34]:
network = Seq2SeqEncDec(len(Vs),len(Vd),128).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(network.parameters())
num_epochs = 25
mb_size = 65

for epoch in range(num_epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_mb = Y_padded_input_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_padded_label_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_label_mb.reshape(Y_label_mb.shape[0]*Y_label_mb.shape[1],)
        
        X_train_mb, Y_input_mb, Y_label_mb = X_train_mb.to(device), Y_input_mb.to(device), Y_label_mb.to(device)

        y_hat_train_mb = network(X_train_mb,Y_input_mb)
        y_hat_train_mb = y_hat_train_mb[0]
        y_hat_train_mb = y_hat_train_mb.reshape(y_hat_train_mb.shape[0]*y_hat_train_mb.shape[1],
                                                y_hat_train_mb.shape[2])

        loss_fn_value = loss_fn(y_hat_train_mb,Y_label_mb)

        loss_fn_value.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(),max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

        print("Epoch # {}, Time Step # {}, Loss Value = {}".format(epoch,i,loss_fn_value))

Epoch # 0, Time Step # 0, Loss Value = 8.863900184631348
Epoch # 0, Time Step # 1, Loss Value = 8.86389446258545
Epoch # 0, Time Step # 2, Loss Value = 8.86389446258545
Epoch # 0, Time Step # 3, Loss Value = 8.863889694213867
Epoch # 0, Time Step # 4, Loss Value = 8.863885879516602
Epoch # 0, Time Step # 5, Loss Value = 8.863877296447754
Epoch # 0, Time Step # 6, Loss Value = 8.863875389099121
Epoch # 0, Time Step # 7, Loss Value = 8.863865852355957
Epoch # 0, Time Step # 8, Loss Value = 8.86386489868164
Epoch # 0, Time Step # 9, Loss Value = 8.863849639892578
Epoch # 0, Time Step # 10, Loss Value = 8.863844871520996
Epoch # 0, Time Step # 11, Loss Value = 8.863829612731934
Epoch # 0, Time Step # 12, Loss Value = 8.863789558410645
Epoch # 0, Time Step # 13, Loss Value = 8.863789558410645
Epoch # 0, Time Step # 14, Loss Value = 8.863743782043457
Epoch # 0, Time Step # 15, Loss Value = 8.863717079162598
Epoch # 0, Time Step # 16, Loss Value = 8.863648414611816
Epoch # 0, Time Step # 17, 